<a href="https://colab.research.google.com/github/DiploDatos/AprendizajeProfundo/blob/add_2024_content/A3_Procesamiento_de_Lenguaje_Natural_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade torch torchtext


In [ ]:
import nltk
import pandas as pd
import torch

from gensim import models
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, SnowballStemmer,WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import gensim.downloader as api

#import torchtext
#from torchtext.vocab import GloVe

In [ ]:
nltk.download('punkt') #Descargamos el tokenizador llamado punkt
nltk.download("stopwords") # Descargamos las stop words
nltk.download('snowball_data') # Algoritmo Porter para stemming version 2
nltk.download('wordnet') # Necesaria para la Lematization

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package snowball_data to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
text_example ="""
Natural language processing (NLP) is an interdisciplinary subfield of computer science and linguistics.
It is primarily concerned with giving computers the ability to support and manipulate speech.
It involves processing natural language datasets, such as text corpora or speech corpora, using either rule-based or probabilistic (i.e. statistical and, most recently, neural network-based) machine learning approaches."""
text_example

'\nNatural language processing (NLP) is an interdisciplinary subfield of computer science and linguistics.\nIt is primarily concerned with giving computers the ability to support and manipulate speech.\nIt involves processing natural language datasets, such as text corpora or speech corpora, using either rule-based or probabilistic (i.e. statistical and, most recently, neural network-based) machine learning approaches.'

## Tokenizacion

In [ ]:
sentences_lst = sent_tokenize(text_example)
sentences_lst

['\nNatural language processing (NLP) is an interdisciplinary subfield of computer science and linguistics.',
 'It is primarily concerned with giving computers the ability to support and manipulate speech.',
 'It involves processing natural language datasets, such as text corpora or speech corpora, using either rule-based or probabilistic (i.e.',
 'statistical and, most recently, neural network-based) machine learning approaches.']

In [ ]:
words_lst = word_tokenize(text_example)
words_lst

['Natural',
 'language',
 'processing',
 '(',
 'NLP',
 ')',
 'is',
 'an',
 'interdisciplinary',
 'subfield',
 'of',
 'computer',
 'science',
 'and',
 'linguistics',
 '.',
 'It',
 'is',
 'primarily',
 'concerned',
 'with',
 'giving',
 'computers',
 'the',
 'ability',
 'to',
 'support',
 'and',
 'manipulate',
 'speech',
 '.',
 'It',
 'involves',
 'processing',
 'natural',
 'language',
 'datasets',
 ',',
 'such',
 'as',
 'text',
 'corpora',
 'or',
 'speech',
 'corpora',
 ',',
 'using',
 'either',
 'rule-based',
 'or',
 'probabilistic',
 '(',
 'i.e',
 '.',
 'statistical',
 'and',
 ',',
 'most',
 'recently',
 ',',
 'neural',
 'network-based',
 ')',
 'machine',
 'learning',
 'approaches',
 '.']

## Stop words

In [ ]:
nltk.download("stopwords") # Descargamos las stop words
stop_words_english = set(stopwords.words("english"))
print(f"Cantidad de stop words en ingles: {len(stop_words_english)}")
print(f"Algunos ejemplos: {list(stop_words_english)[:20]}")

Cantidad de stop words en ingles: 179
Algunos ejemplos: ["should've", 'himself', 'being', 'at', 'nor', 'from', 'as', 'haven', 'ain', 'any', 'hasn', 'll', "shouldn't", 'a', 'up', 'is', 'or', 'ours', 'down', 'do']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
stop_words_spanish = set(stopwords.words("spanish"))
print(f"Cantidad de stop words en español: {len(stop_words_spanish)}")
print(f"Algunos ejemplos: {list(stop_words_spanish)[:20]}")

Cantidad de stop words en español: 313
Algunos ejemplos: ['mías', 'habrían', 'vuestras', 'hay', 'fuesen', 'una', 'este', 'tendrías', 'estuviste', 'tendrán', 'ni', 'fueron', 'estas', 'antes', 'teníais', 'son', 'tuviera', 'vosotros', 'hubieron', 'suya']


In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words_spanish = set(stopwords.words("english"))
filtered_list_eng = []
for word in words_lst:
  # casefold es una manera de trabajar indistintamente mayusculas y minusculas
  # ya que las stop words están todas en minúscula
   if word.casefold() not in stop_words_english:
        filtered_list_eng.append(word)

print(f"Cantidad de palabras iniciales: {len(words_lst)}")
print(words_lst)
print('\n')
print(f"Cantidad de palabras luego de remover stop words: {len(filtered_list_eng)}")
print(filtered_list_eng)

Cantidad de palabras iniciales: 67
['Natural', 'language', 'processing', '(', 'NLP', ')', 'is', 'an', 'interdisciplinary', 'subfield', 'of', 'computer', 'science', 'and', 'linguistics', '.', 'It', 'is', 'primarily', 'concerned', 'with', 'giving', 'computers', 'the', 'ability', 'to', 'support', 'and', 'manipulate', 'speech', '.', 'It', 'involves', 'processing', 'natural', 'language', 'datasets', ',', 'such', 'as', 'text', 'corpora', 'or', 'speech', 'corpora', ',', 'using', 'either', 'rule-based', 'or', 'probabilistic', '(', 'i.e', '.', 'statistical', 'and', ',', 'most', 'recently', ',', 'neural', 'network-based', ')', 'machine', 'learning', 'approaches', '.']


Cantidad de palabras luego de remover stop words: 50
['Natural', 'language', 'processing', '(', 'NLP', ')', 'interdisciplinary', 'subfield', 'computer', 'science', 'linguistics', '.', 'primarily', 'concerned', 'giving', 'computers', 'ability', 'support', 'manipulate', 'speech', '.', 'involves', 'processing', 'natural', 'language'

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Ejercicio:** Hacer el mismo proceso partiendo de un texto ejemplo en español.

## Stemming

In [ ]:
stemmer = PorterStemmer()

In [ ]:
words_lst[:10]

['Natural',
 'language',
 'processing',
 '(',
 'NLP',
 ')',
 'is',
 'an',
 'interdisciplinary',
 'subfield']

In [ ]:
stemmed_words = [stemmer.stem(word) for word in words_lst]
stemmed_words[:15]

['natur',
 'languag',
 'process',
 '(',
 'nlp',
 ')',
 'is',
 'an',
 'interdisciplinari',
 'subfield',
 'of',
 'comput',
 'scienc',
 'and',
 'linguist']

In [ ]:
examples_words = ['Discovery'	,'discovered'	,'discoveries'	,'Discovering']
#examples_words = ['play'	,'played'	,'plays'	,'playing']
for word in examples_words:
  print(f"Original word: {word}")
  print(f"Stemmed word: {stemmer.stem(word)}")
  print('--------')


Original word: Discovery
Stemmed word: discoveri
--------
Original word: discovered
Stemmed word: discov
--------
Original word: discoveries
Stemmed word: discoveri
--------
Original word: Discovering
Stemmed word: discov
--------


Veamos si hay algún cambio en el resultado cambiando el algoritmo de stemming...

In [ ]:
stemmer = SnowballStemmer("english")
examples_words = ['Discovery'	,'discovered'	,'discoveries'	,'Discovering']
for word in examples_words:
  print(f"Original word: {word}")
  print(f"Stemmed word: {stemmer.stem(word)}")
  print('--------')


Original word: Discovery
Stemmed word: discoveri
--------
Original word: discovered
Stemmed word: discov
--------
Original word: discoveries
Stemmed word: discoveri
--------
Original word: Discovering
Stemmed word: discov
--------


## Lemmatizing

In [ ]:
lemmatizer = WordNetLemmatizer()
examples_words = ['discovery'	,'disco'	,'discoveries'	,'discovering']
for word in examples_words:
  print(f"Original word: {word}")
  print(f"Stemmed word: {lemmatizer.lemmatize(word)}")
  print('--------')

Original word: discovery
Stemmed word: discovery
--------
Original word: disco
Stemmed word: disco
--------
Original word: discoveries
Stemmed word: discovery
--------
Original word: discovering
Stemmed word: discovering
--------


In [ ]:
lemmatizer.lemmatize("worst") #Por defecto toma considera "nouns"

'worst'

El parametro `pos` es por defecto `'n'` donde `n` está asociado a `noun`. Sin embargo, para asegurarnos que `"worst"` sea tratado como un adjetivo  necesitamos cambiar el parámetro por defecto a `pos="a"`.

In [ ]:
lemmatizer.lemmatize("worst", pos="a")

'bad'

## Bag of words

In [ ]:
new_text_example = """
In 2003, word n-gram model, at the time the best statistical algorithm, was overperformed by a multi-layer perceptron (with a single hidden layer and context length of several words trained on up to 14 million of words with a CPU cluster in language modelling) by Yoshua Bengio with co-authors.[8]

In 2010, Tomáš Mikolov (then a PhD student at Brno University of Technology) with co-authors applied a simple recurrent neural network with a single hidden layer to language modelling,=[9] and in the following years he went on to develop Word2vec. In the 2010s, representation learning and deep neural network-style (featuring many hidden layers) machine learning methods became widespread in natural language processing."""

In [ ]:
sentences = sent_tokenize(new_text_example)
sentences

['\nIn 2003, word n-gram model, at the time the best statistical algorithm, was overperformed by a multi-layer perceptron (with a single hidden layer and context length of several words trained on up to 14 million of words with a CPU cluster in language modelling) by Yoshua Bengio with co-authors.',
 '[8]\n\nIn 2010, Tomáš Mikolov (then a PhD student at Brno University of Technology) with co-authors applied a simple recurrent neural network with a single hidden layer to language modelling,=[9] and in the following years he went on to develop Word2vec.',
 'In the 2010s, representation learning and deep neural network-style (featuring many hidden layers) machine learning methods became widespread in natural language processing.']

In [ ]:
len(sentences)

3

In [ ]:
count_vectorizer = CountVectorizer()
X = count_vectorizer.fit_transform(sentences)
X = X.toarray()
X

array([[1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 2, 1, 1, 1, 1, 0, 0, 0, 0,
        1, 0, 1, 2, 1, 2, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 2, 1,
        1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 2, 0, 1, 1, 0, 1, 0, 1, 1,
        0, 0, 3, 1, 0, 2, 0, 1],
       [0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1,
        0, 1, 1, 2, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1,
        0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 2, 1, 0, 1, 0, 0,
        1, 0, 2, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
        0, 0, 1, 2, 1, 0, 1, 2, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
        0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0]])

In [ ]:
X.shape

(3, 74)

In [ ]:
count_vectorizer.vocabulary_.keys()


dict_keys(['in', '2003', 'word', 'gram', 'model', 'at', 'the', 'time', 'best', 'statistical', 'algorithm', 'was', 'overperformed', 'by', 'multi', 'layer', 'perceptron', 'with', 'single', 'hidden', 'and', 'context', 'length', 'of', 'several', 'words', 'trained', 'on', 'up', 'to', '14', 'million', 'cpu', 'cluster', 'language', 'modelling', 'yoshua', 'bengio', 'co', 'authors', '2010', 'tomáš', 'mikolov', 'then', 'phd', 'student', 'brno', 'university', 'technology', 'applied', 'simple', 'recurrent', 'neural', 'network', 'following', 'years', 'he', 'went', 'develop', 'word2vec', '2010s', 'representation', 'learning', 'deep', 'style', 'featuring', 'many', 'layers', 'machine', 'methods', 'became', 'widespread', 'natural', 'processing'])

In [ ]:
count_vectorizer2 = CountVectorizer(ngram_range=(2,2))
X2 = count_vectorizer2.fit_transform(sentences)
X2 = X2.toarray()
X2

array([[1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
        0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0,
        0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1,
        0, 0, 1, 1, 1, 1, 1, 1, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
        0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1,
        0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0,
        1, 0, 1, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0,
        1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
X2.shape

(3, 98)

In [ ]:
sorted(count_vectorizer2.vocabulary_.keys())


['14 million',
 '2003 word',
 '2010 tomáš',
 '2010s representation',
 'algorithm was',
 'and context',
 'and deep',
 'and in',
 'applied simple',
 'at brno',
 'at the',
 'authors applied',
 'became widespread',
 'bengio with',
 'best statistical',
 'brno university',
 'by multi',
 'by yoshua',
 'cluster in',
 'co authors',
 'context length',
 'cpu cluster',
 'deep neural',
 'develop word2vec',
 'featuring many',
 'following years',
 'gram model',
 'he went',
 'hidden layer',
 'hidden layers',
 'in 2003',
 'in 2010',
 'in language',
 'in natural',
 'in the',
 'language modelling',
 'language processing',
 'layer and',
 'layer perceptron',
 'layer to',
 'layers machine',
 'learning and',
 'learning methods',
 'length of',
 'machine learning',
 'many hidden',
 'methods became',
 'mikolov then',
 'million of',
 'model at',
 'modelling and',
 'modelling by',
 'multi layer',
 'natural language',
 'network style',
 'network with',
 'neural network',
 'of several',
 'of technology',
 'of words

## TF- IDF

La intuición detrás de esto es que cuanto más común es una palabra en todos los documentos, menor es su importancia para el documento actual.

In [ ]:
sentences

['\nIn 2003, word n-gram model, at the time the best statistical algorithm, was overperformed by a multi-layer perceptron (with a single hidden layer and context length of several words trained on up to 14 million of words with a CPU cluster in language modelling) by Yoshua Bengio with co-authors.',
 '[8]\n\nIn 2010, Tomáš Mikolov (then a PhD student at Brno University of Technology) with co-authors applied a simple recurrent neural network with a single hidden layer to language modelling,=[9] and in the following years he went on to develop Word2vec.',
 'In the 2010s, representation learning and deep neural network-style (featuring many hidden layers) machine learning methods became widespread in natural language processing.']

In [ ]:
tfidf = TfidfVectorizer()
transformed = tfidf.fit_transform(sentences)

In [ ]:
transformed[0]

<1x74 sparse matrix of type '<class 'numpy.float64'>'
	with 40 stored elements in Compressed Sparse Row format>

In [ ]:
df = pd.DataFrame(transformed[0].T.todense(),
    	index=tfidf.get_feature_names_out(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
df.shape

(74, 1)

In [ ]:
df.sort_values(by=['TF-IDF'], ascending=False)

,TF-IDF
with,0.326879
words,0.286538
by,0.286538
layer,0.217920
of,0.217920
...,...
tomáš,0.000000
university,0.000000
word2vec,0.000000
he,0.000000


## Embeddings

Algunos links interesantes:

- https://ronxin.github.io/wevi/

### Word2Vec

In [ ]:
path = api.load("word2vec-google-news-300", return_path=True)
print(path)

[==================================================] 100.0% 1662.8/1662.8MB downloaded
/root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz


In [ ]:
w2v = models.KeyedVectors.load_word2vec_format('/root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz', binary=True)

In [ ]:
vect = w2v['worst']
print(vect.shape)
print(vect)

(300,)
[ 1.66015625e-01 -8.20312500e-02  4.51171875e-01  1.22558594e-01
  1.34765625e-01  1.33789062e-01 -3.88671875e-01 -3.66210938e-02
  5.15625000e-01  1.04003906e-01  1.82617188e-01 -1.66015625e-01
  2.00195312e-01  2.12890625e-01 -1.90429688e-02  8.74023438e-02
 -7.75146484e-03 -2.87109375e-01 -1.54296875e-01 -3.86718750e-01
  2.15820312e-01  2.04101562e-01 -3.14453125e-01 -4.00390625e-01
  2.57812500e-01  3.17382812e-02  5.73730469e-03  1.16699219e-01
  2.20703125e-01  1.01074219e-01 -1.53320312e-01 -3.83300781e-02
  1.38549805e-02  9.22851562e-02 -1.68457031e-02  2.27539062e-01
  1.07421875e-01 -2.43164062e-01 -9.81445312e-02  3.49609375e-01
  1.97265625e-01 -1.80664062e-01 -4.27246094e-02 -2.09960938e-01
  1.14257812e-01  4.37500000e-01  1.00585938e-01  5.22460938e-02
 -1.41601562e-02  1.16210938e-01  1.32446289e-02  2.73437500e-01
 -1.41906738e-03  3.30078125e-01 -8.34960938e-02 -2.49023438e-01
 -1.33789062e-01 -1.84570312e-01  8.59375000e-02  2.48046875e-01
 -1.74804688e-01  

In [ ]:
w2v.most_similar('worst')


[('Worst', 0.6146091818809509),
 ('weakest', 0.6143776774406433),
 ('scariest', 0.5957257747650146),
 ('ugliest', 0.5931181311607361),
 ('best', 0.5835111141204834),
 ('bleakest', 0.5718506574630737),
 ('strongest', 0.567145586013794),
 ('nastiest', 0.5644308924674988),
 ('lousiest', 0.563145101070404),
 ('toughest', 0.5624396204948425)]

In [ ]:
sentences = [sent for sent in sentences]
custom_model = models.Word2Vec(sentences, min_count=1,workers=4)
custom_model


In [ ]:

# Supongamos que tienes una lista de oraciones donde cada oración es una lista de palabras.
sentences = [["esto", "es", "una", "oración"], ["esto", "es", "otra", "oración"]]

# Entrenar un modelo Word2Vec
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, sg=0)

In [ ]:
# Obtener el embedding de una palabra
embedding = model.wv["esto"]
embedding

array([ 9.4563962e-05,  3.0773198e-03, -6.8126451e-03, -1.3754654e-03,
        7.6685809e-03,  7.3464094e-03, -3.6732971e-03,  2.6427018e-03,
       -8.3171297e-03,  6.2054861e-03, -4.6373224e-03, -3.1641065e-03,
        9.3113566e-03,  8.7338570e-04,  7.4907029e-03, -6.0740625e-03,
        5.1605068e-03,  9.9228229e-03, -8.4573915e-03, -5.1356913e-03,
       -7.0648370e-03, -4.8626517e-03, -3.7785638e-03, -8.5361991e-03,
        7.9556061e-03, -4.8439382e-03,  8.4236134e-03,  5.2625705e-03,
       -6.5500261e-03,  3.9578713e-03,  5.4701497e-03, -7.4265362e-03,
       -7.4057197e-03, -2.4752307e-03, -8.6257253e-03, -1.5815723e-03,
       -4.0343284e-04,  3.2996845e-03,  1.4418805e-03, -8.8142155e-04,
       -5.5940580e-03,  1.7303658e-03, -8.9737179e-04,  6.7936908e-03,
        3.9735902e-03,  4.5294715e-03,  1.4343059e-03, -2.6998555e-03,
       -4.3668128e-03, -1.0320747e-03,  1.4370275e-03, -2.6460087e-03,
       -7.0737829e-03, -7.8053069e-03, -9.1217868e-03, -5.9351693e-03,
      

### Glove

Algunos links útiles:


- https://notebook.community/spro/practical-pytorch/glove-word-vectors/glove-word-vectors

In [ ]:
glove = vocab.GloVe(name='6B', dim=100)

print('Loaded {} words'.format(len(glove.itos)))

NameError: name 'vocab' is not defined

In [ ]:
def get_word(word):
    return glove.vectors[glove.stoi[word]]

In [ ]:
def closest(vec, n=10):
    """
    Find the closest words for a given vector
    """
    all_dists = [(w, torch.dist(vec, get_word(w))) for w in glove.itos]
    return sorted(all_dists, key=lambda t: t[1])[:n]

In [ ]:
def print_tuples(tuples):
    for tuple in tuples:
        print('(%.4f) %s' % (tuple[1], tuple[0]))

In [ ]:
get_word('google')

In [ ]:
print_tuples(closest(get_word('google')))


In [ ]:
print_tuples(closest(get_word('science')))
